In [65]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm
from jcopdl.callback import Callback

version = 3

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier[1] = torch.nn.Linear(model.last_channel, 3)
model.classifier

Using cache found in /Users/yangjunhui/.cache/torch/hub/pytorch_vision_v0.10.0


Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [66]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [67]:
bs = 32
DATA_DIR = f"data/single_classifier_data_v{version}"

train_set = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True)

val_set = datasets.ImageFolder(f"{DATA_DIR}/valid", transform=test_transform)
valloader = DataLoader(val_set, batch_size=bs, shuffle=True)

test_set = datasets.ImageFolder(f"{DATA_DIR}/test", transform=test_transform)
testloader = DataLoader(test_set, shuffle=True)

In [68]:
config = {
    'batch_size': bs,
    'output_size': len(train_set.classes)
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
callback = Callback(model, config, early_stop_patience=3, outdir='model')

In [69]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == 'train':
        model.train()
    elif mode == 'val':
        model.eval()

    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)

        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()   

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
        
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [70]:
while True:
    train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn('val', val_set, valloader, model, criterion, optimizer, device)

    # Logging
    callback.log("train_cost", train_cost)
    callback.log("test_cost", test_cost)
    callback.log("train_score", train_score)
    callback.log("test_score", test_score)

    # Early Stopping
    if callback.early_stopping("maximize", monitor='test_score'):
        callback.add_plot(["train_cost", "test_cost"], scale="semilogy")
        callback.add_plot(["train_score", "test_score"], scale="semilogy")
        break

Val: 100%|████████████████████████████████████████| 5/5 [00:02<00:00,  1.77it/s]


,train_cost,test_cost,train_score,test_score
epoch,,,,
1,-0.198294,-0.453653,0.422500,0.486667
2,-0.674812,-0.988025,0.524167,0.553333
3,-1.242521,-1.683970,0.571667,0.586667
4,-1.901341,-2.532506,0.618333,0.633333
5,-2.780396,-3.543403,0.650000,0.653333
6,-3.705114,-4.638940,0.698333,0.673333
7,-4.698391,-5.457698,0.695000,0.733333
8,-5.627955,-6.412279,0.735000,0.740000
9,-6.575008,-7.434821,0.748333,0.773333


Val: 100%|████████████████████████████████████████| 5/5 [00:02<00:00,  1.99it/s]


==> Execute Early Stopping at epoch: 36 | Best test_score: 0.9733
==> Best model is saved at model


In [71]:
test_cost, test_score = loop_fn('test', test_set, testloader, model, criterion, optimizer, device)
test_score

Test: 100%|███████████████████████████████████| 150/150 [00:03<00:00, 39.32it/s]


0.9533333333333334

In [72]:
import os
import shutil
from pathlib import Path
from PIL import Image

def classify_image(source_dir, image_name, dest_dir):
    path = os.path.join(source_dir, image_name)
    image = Image.open(Path(path))
    input = test_transform(image)
    input = input.view(1, 3, 128, 128)

    output = model(input)
    prediction = int(torch.max(output.data, 1)[1].numpy())
    
    if (prediction == 0):
        dest = f"{dest_dir}/mild/"
    elif (prediction == 1):
        dest = f"{dest_dir}/moderate/"
    elif (prediction == 2):
        dest = f"{dest_dir}/severe/"
    dest += image_name
        
    shutil.copyfile(path, dest)

In [79]:
import os
import glob
from tqdm import tqdm

scene_num = 2
source_dir = f"../potholes_evaluation/scene{scene_num}/imgs"
dest_dir = f"../potholes_evaluation/scene{scene_num}/results"

os.makedirs(f'{dest_dir}/mild')
os.makedirs(f'{dest_dir}/moderate')
os.makedirs(f'{dest_dir}/severe')

pathlist = Path(source_dir).glob('**/*.png')
for path in tqdm(pathlist):
    image_name = str(path).split("/")[-1]
    classify_image(source_dir, image_name, dest_dir)

660it [00:19, 33.56it/s]


In [80]:
import csv

f = open(f'{dest_dir}/results.csv', 'w')
writer = csv.writer(f)
writer.writerow(["image", "type"])

images = os.listdir(source_dir)
mild_images = os.listdir(f"{dest_dir}/mild")
moderate_images = os.listdir(f"{dest_dir}/moderate")
severe_images = os.listdir(f"{dest_dir}/severe")

for image in images:
    if image in mild_images:
        writer.writerow([image, "mild"])
    elif image in moderate_images:
        writer.writerow([image, "moderate"])
    else:
        writer.writerow([image, "severe"])
        